In [ ]:
# !find ./log/* -type f -not -name "stepNum*" -exec rm -f {} \;
# clean the json format log and it's corresponding csv file

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json
import matplotlib
# 设置字体为 Times New Roman
matplotlib.rcParams['font.family'] = 'Times New Roman'
# 配色设置
COLORS = ['red', 'blue', 'green', '#dcbeff', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
          'olive', 'lime', 'teal', 'navy']
# COLORS = plt.get_cmap('tab20c').COLORS
# cite: https://sashamaps.net/docs/resources/20-COLORS/ 最好分辨的20种颜色
# COLORS = ['#e6194B', '#3cb44b', '#f58231', 'blue', '#ffe119', '#f032e6', '#42d4f4', '#fabed4', '#469990', 'purple', '#9A6324', '#000075', '#800000', '#aaffc3', '#fffac8', '#a9a9a9', '#ffffff', '#000000']
print("COLORS_len:", len(COLORS))
plt.figure(figsize=(10, 1))
for i in range(len(COLORS)):
    plt.fill_between([i, i + 1], 0, 1, color=COLORS[i])
plt.show()
# 作图设置
LABLE_SIZE = 30
TICK_SIZE = 25
SHOW_LEGEND = False

# 文件导出位置
SUB_DIR = "long_vs_short/"

In [ ]:
class Log():
    def __init__(self, json_log_path):
        self.json_log_path = json_log_path
        self.param = json.load(open(self.json_log_path, 'r'))
        self.param_names = list(self.param.keys())
    def __getitem__(self, key):
        assert key in self.param.keys(), "key not in param"
        return self.param[key]
    
    def get_data(self,nrows=None):
        if nrows is None:
            return pandas.read_csv(self.param["data"])
        else:
            return pandas.read_csv(self.param["data"], nrows=nrows)
    
    def __str__(self) -> str:
        return f"Log({self.json_log_path})"
    
    def serialize(self) -> str:
        """
        序列化的时候忽略属性"other"
        """
        temp_json = self.param.copy()
        temp_json.pop("other")
        temp_json.pop("data")
        return json.dumps(temp_json)
    

# log1 = Log("./log/20231224063327_11cd96b4-ca81-4e25-ad9e-5aec0c3f3fa3.json")
# print(log1.param_names, log1["normId"])
# log1["data"]
# print(log1.serialize())
# log1.get_data()
# del log1

In [ ]:
import pandas
import os

NROWS = 200000
end = NROWS
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

In [ ]:
# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
longterm_logs = list()
for log in logs:
    if log["stepNum"] == 50000 and log['mu'] == 0.001 and log['p0'] == 0.5 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error" and log["gamma"] == 1:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    # if log["stepNum"] == 100000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
        # 时间
        time_str = log['data'].split("/")[-1].split("_")[0]
        if int(time_str[:8]) >= 20240321:
            print(log.serialize())
            longterm_logs.append(log)
    LOG_STEP = log['other']['logStep']
print("longterm_logs_num:", len(longterm_logs))

In [ ]:
# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
shortterm_logs = list()
for log in logs:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    if log["stepNum"] == 50000 and log["mu"] == 0.001 and log["p0"] == 0.5 and log["payoffMatrix"] == "payoffMatrix_shortterm":
    # if log["stepNum"] == 100000 and log["mu"] == 0.001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_shortterm":
        time_str = log['data'].split("/")[-1].split("_")[0]
        if int(time_str[:8]) >= 20240321:
            print(log.serialize())
            shortterm_logs.append(log)
print("shortterm_logs_num:", len(shortterm_logs))

In [ ]:
type2logs = {
    "shortterm": shortterm_logs,
    "longterm": longterm_logs
}

In [ ]:
# 画第25列
has_legend = True
# plot_norm_lst = list(range(16))
plot_norm_lst = [4,6,9,11]
print("plot_norm_lst:", plot_norm_lst)

for key, logs in type2logs.items():
    fig = plt.figure(figsize=(10, 5))
    color_index = 0
    plt.xlabel("Time")
    for log in logs:
        if log['normId'] not in plot_norm_lst:
            continue

        # 读取数据
        data = log.get_data(NROWS)
        # 第一列为横轴，第二列为纵轴画曲线，label为列名
        # plt.figure(figsize=(20, 6))
        # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.subplots_adjust(top=0.8)
        
        i = 26
        index = i - 25
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], color=COLORS[color_index], label="norm" + str(log["normId"]))
        color_index += 1
        # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
        # plt.title()
        # plt.savefig('./norm10-c-ur.png', dpi=600)

    # 刻度间隔
    # x_major_locator = plt.MultipleLocator(5000)
    # y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    # ax.xaxis.set_major_locator(x_major_locator)
    # ax.yaxis.set_major_locator(y_major_locator)
    margin = 0.02
    plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
    plt.ylim(-1 * margin * 2, 1 + margin * 2)
    plt.xticks(fontsize=TICK_SIZE)
    plt.yticks(fontsize=TICK_SIZE)
    plt.xlabel("Time", fontsize=LABLE_SIZE)
    plt.ylabel("Cooperation rate", fontsize=LABLE_SIZE)
    # plt.tight_layout()
    if has_legend:
        plt.legend(fontsize=TICK_SIZE - 8)
        # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True, fontsize=TICK_SIZE - 8)
    plt.tight_layout()
    plt.savefig('./temp_fig/' + SUB_DIR + 'cr_all_norm_' + key + '.pdf', dpi=600, format='pdf', bbox_inches='tight')
    plt.savefig('./temp_fig/' + SUB_DIR + 'cr_all_norm_' + key + '.png', dpi=600, format='png', bbox_inches='tight')

# 留存图例
fig = plt.figure()
handles, labels = ax.get_legend_handles_labels()
leg = fig.legend(handles, labels, loc='center', fontsize=LABLE_SIZE, ncol = 8)
for line in leg.get_lines():
    line.set_linewidth(4.0)

fig.savefig('./temp_fig/' + SUB_DIR + 'cr_norm' + '_legend' + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
fig.savefig('./temp_fig/' + SUB_DIR + 'cr_norm' + '_legend' + '.png', dpi=600, bbox_inches='tight', format="png")
plt.close(fig)   

In [ ]:
# 画第25列
has_legend = True
# plot_norm_lst = list(range(16))
plot_norm_lst = [4,6,9,11]
print("plot_norm_lst:", plot_norm_lst)

for key, logs in type2logs.items():
    fig = plt.figure(figsize=(10, 5))
    color_index = 0
    plt.xlabel("Time")
    for log in logs:
        if log['normId'] not in plot_norm_lst:
            continue
        # 读取数据
        data = log.get_data(NROWS)
        # 第一列为横轴，第二列为纵轴画曲线，label为列名
        # plt.figure(figsize=(20, 6))
        # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.subplots_adjust(top=0.8)
        
        i = 25
        index = i - 25
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], color=COLORS[color_index], label="norm" + str(log["normId"]))
        color_index += 1
        # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
        # plt.title()
        # plt.savefig('./norm10-c-ur.png', dpi=600)

    # 刻度间隔
    # x_major_locator = plt.MultipleLocator(5000)
    # y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    # ax.xaxis.set_major_locator(x_major_locator)
    # ax.yaxis.set_major_locator(y_major_locator)
    margin = 0.02
    plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
    plt.ylim(-1 * margin * 2, 1 + margin * 2)
    plt.xticks(fontsize=TICK_SIZE)
    plt.yticks(fontsize=TICK_SIZE)
    plt.xlabel("Time", fontsize=LABLE_SIZE)
    plt.ylabel("Good reputation", fontsize=LABLE_SIZE)
    # plt.tight_layout()
    if has_legend:
        # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True, fontsize=TICK_SIZE - 8)
        plt.legend(fontsize=TICK_SIZE - 8)
    plt.tight_layout()
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_all_norm_' + key + '.pdf', dpi=600, format='pdf', bbox_inches='tight')
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_all_norm_' + key + '.png', dpi=600, format='png', bbox_inches='tight')

# 留存图例
fig = plt.figure()
handles, labels = ax.get_legend_handles_labels()
leg = fig.legend(handles, labels, loc='center', fontsize=LABLE_SIZE, ncol = 8)
for line in leg.get_lines():
    line.set_linewidth(4.0)

fig.savefig('./temp_fig/' + SUB_DIR + 'good_rep_norm' + '_legend' + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
fig.savefig('./temp_fig/' + SUB_DIR + 'good_rep_norm' + '_legend' + '.png', dpi=600, bbox_inches='tight', format="png")
plt.close(fig)   

In [ ]:
# 画第25列
log_len = len(type2logs["shortterm"])


for idx in range(log_len):
    fig, axes = plt.subplots(1,2,figsize=(20, 5))
    color_index = 0
    for type_name in type2logs.keys():
        log = type2logs[type_name][idx]
        # 读取数据
        data = log.get_data(NROWS)
        # 第一列为横轴，第二列为纵轴画曲线，label为列名
        # plt.figure(figsize=(20, 6))
        # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
        # fig.suptitle("Norm Id: " + str(log["normId"]), fontsize=LABLE_SIZE)
        # fig.supxlabel("Time", fontsize=LABLE_SIZE)
        # fig.subplots_adjust(top=0.8)
        for i in range(25, 27):
            index = i - 25
            if (data.columns.values[i] == 'good_rep'):
                # axes[index].set_xlabel(data.columns.values[0])
                axes[index].set_ylabel("Good reputation", fontsize=LABLE_SIZE)
            elif (data.columns.values[i] == 'cr'):
                axes[index].set_ylabel("Cooperation rate", fontsize=LABLE_SIZE)
            else: 
                raise "error"

            if type_name == "shortterm":
                label_name = "Short-term"
            else:
                label_name = "Long-term"
            axes[index].plot(data.iloc[:end, 0], data.iloc[:end, i], label=label_name, color=COLORS[color_index])
            
            axes[index].legend(fontsize=TICK_SIZE - 8, loc='center right')
            axes[index].set_xlabel('Time', fontsize=LABLE_SIZE)
            axes[index].xaxis.set_tick_params(labelsize=TICK_SIZE)
            axes[index].yaxis.set_tick_params(labelsize=TICK_SIZE)
        # legend
        # axes[index].legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True, fontsize=TICK_SIZE - 8)
        color_index += 1
        fig.tight_layout()
        
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_cr_norm' + str(log['normId']) + '.pdf', dpi=600, bbox_inches='tight', format='pdf')
    plt.savefig('./temp_fig/' + SUB_DIR + 'good_rep_cr_norm' + str(log['normId']) + '.png', dpi=600, bbox_inches='tight', format='png')


In [ ]:
import numpy as np

# 画第25列
# 计算一定区间内的平均合作率  以 norm 为 x 画出柱状图
range_start = 10000
range_end = 50000

fig = plt.figure(figsize=(10, 6))
N = 16
type2mean_cr = dict()
for key, logs in type2logs.items():
    for log in logs:
        data = log.get_data(NROWS)
        # print(data.iloc[:, 26])
        # 取出 stepNum 在 range_start 和 range_end 之间的数据
        filtered_data = data[data['step'].between(range_start, range_end)]
        cr_series = filtered_data.iloc[:, 26]
        if key not in type2mean_cr.keys():
            type2mean_cr[key] = list()
        type2mean_cr[key].append(cr_series.mean())
    print(f"len: {len(type2mean_cr[key])}")


# 画n条并列柱状图
# 并列条数
bar_num = len(type2mean_cr.keys())
colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
width = 1 / (bar_num + 2)

ind = np.arange(N)
# 根据 bar_num 调整 first_ind
ind = ind - (bar_num+1) * width / 2

color_id = 0
for key, mean_cr in type2mean_cr.items():
    ind += width
    print(f"key: {key} value: {mean_cr}")
    # plt.bar(ind, mean_cr, width, label='mean_cr', color=colors[color_id])
    print(color_id)
    b = 4
    # 开启 latex 渲染
    plt.bar(ind, mean_cr, width, label=key, color=COLORS[color_id])
    color_id += 1

# 刻度间隔
y_major_locator = plt.MultipleLocator(0.1)
ax = plt.gca()
ax.yaxis.set_major_locator(y_major_locator)
margin = 0.02
# plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
plt.xticks(range(0, N), fontsize=TICK_SIZE)
plt.yticks(fontsize=TICK_SIZE)


plt.ylim(-1 * margin * 2, 1 + margin * 2)
plt.xlabel("Norm id", fontsize=LABLE_SIZE)
plt.ylabel("Average cooperation rate", fontsize=LABLE_SIZE)
plt.tight_layout()
plt.savefig('./cr_all_norm.png', dpi=600)
plt.legend(fontsize=LABLE_SIZE - 10)
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=8, fancybox=True, shadow=True)
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.10), ncol=8, fancybox=True, shadow=True, fontsize=TICK_SIZE)
plt.tight_layout()
plt.savefig('./temp_fig/' + 'cr_all_norm' + '.png', dpi=600, format='png', bbox_inches='tight')
plt.savefig('./temp_fig/' + 'cr_all_norm' + '.pdf', dpi=600, format='pdf', bbox_inches='tight')

In [ ]:
# !jupyter nbconvert --to markdown visualize_json_longterm.ipynb

In [ ]:
# # 用pandoc 转为 pdf
# !pandoc visualize_json_longterm.md -o visualize_json_longterm.pdf --pdf-engine=xelatex  -V geometry:margin=1in 